# RA data


TODO:

(- remove duplicated columns, e.g. $"lpk_ver_nr" === $"ver_nr")<br>




In [1]:
spark.catalog.listTables.show(false)

+------------------+--------+-----------+---------+-----------+
|name              |database|description|tableType|isTemporary|
+------------------+--------+-----------+---------+-----------+
|raw_tarif_init    |default |null       |EXTERNAL |false      |
|raw_tarif_update  |default |null       |EXTERNAL |false      |
|stg_aufenthaltsart|default |null       |EXTERNAL |false      |
|stg_deckung       |default |null       |EXTERNAL |false      |
|stg_ecdetail      |default |null       |EXTERNAL |false      |
|stg_eckopf        |default |null       |EXTERNAL |false      |
|stg_ecprodukt     |default |null       |EXTERNAL |false      |
|stg_familie       |default |null       |EXTERNAL |false      |
|stg_gemeinde      |default |null       |EXTERNAL |false      |
|stg_leipkopf      |default |null       |EXTERNAL |false      |
|stg_leippos       |default |null       |EXTERNAL |false      |
|stg_schadenart    |default |null       |EXTERNAL |false      |
|stg_tarif_init    |default |null       

In [2]:
spark.table("default.stg_LeiPPos").printSchema()

root
 |-- lpp_lpk_id: integer (nullable = true)
 |-- lpp_id: integer (nullable = true)
 |-- lpp_typ: integer (nullable = true)
 |-- lpp_lpl_id: short (nullable = true)
 |-- lpp_sca_id: integer (nullable = true)
 |-- lpp_bha_id: string (nullable = true)
 |-- lpp_ktk_id: integer (nullable = true)
 |-- lpp_kma_id: integer (nullable = true)
 |-- lpp_lsa_id: short (nullable = true)
 |-- lpp_dkg_id: short (nullable = true)
 |-- lpp_dgp_id: integer (nullable = true)
 |-- lpp_vde_card: integer (nullable = true)
 |-- lpp_vde_id: integer (nullable = true)
 |-- lpp_vde_pk: integer (nullable = true)
 |-- lpp_beh_beg_dat: timestamp (nullable = true)
 |-- lpp_pos_betr: decimal(19,4) (nullable = true)
 |-- lpp_npl_id: short (nullable = true)
 |-- lpp_np_text: string (nullable = true)
 |-- lpp_np_betr: decimal(19,4) (nullable = true)
 |-- lpp_rl_betr: decimal(19,4) (nullable = true)
 |-- lpp_fr_betr: decimal(10,4) (nullable = true)
 |-- lpp_fr_tot_betr: decimal(10,4) (nullable = true)
 |-- lpp_sb_betr

In [5]:
spark.table("default.stg_LeiPPos").count()

1127420

In [3]:
import java.time.LocalDate
import java.time.format.DateTimeFormatter

val dayFormat = DateTimeFormatter.ofPattern("yyyy-MM-dd")
val rangeStart = LocalDate.parse("2020-01-01", dayFormat)
val tmp = ""
//if (tmp == "") {
  val rangeEnd = rangeStart.plusYears(1).minusDays(1)
//} else {
//  val rangeEnd = LocalDate.parse(tmp, dayFormat)
//}


val df_leiPPos = spark.table("default.stg_LeiPPos").as("leippos")
val df_leiPKopf = spark.table("default.stg_LeiPKopf").as("leipkopf")
val df_vers = spark.table("default.stg_Versicherter")
val df_familie = spark.table("default.stg_Familie")
val df_gemeinde = spark.table("default.stg_Gemeinde")
val df_ecKopf = spark.table("default.stg_ECKopf")
val df_ecDetail = spark.table("default.stg_ECDetail")
val df_ecProdukt = spark.table("default.stg_ECProdukt")

val join_type = "left" // "left_semi" would only take rows which match

val df = df_leiPPos
      // Jahr
      .filter($"LPP_BEH_BEG_DAT" > rangeStart && $"LPP_BEH_BEG_DAT" < rangeEnd)
      
      // AHV-Nr: LeiPPos.LPP_LPK_ID → LeiPKopf.LPK_VER_NR → Versicherter.VER_NNSS_NR
      .join(df_leiPKopf, $"lpp_lpk_id" === $"lpk_id", join_type)
      .join(df_vers, $"lpk_ver_nr" === $"ver_nr", join_type)
      .withColumnRenamed("ver_nnss_nr", "AHV_Nr")

      // Wohnkanton: LeiPPos.LPP_LPK_ID → LeiPKopf.LPK_VER_NR → Versicherter.VER_FAM_NR → Familie.FAM_RA_GDE_ID (Test auf Zeitraum) → Gemeinde.GDE_KT
      .join(df_familie, $"ver_fam_nr" === $"fam_nr", join_type)
      .join(df_gemeinde, $"fam_ra_gde_id" === $"gde_id", join_type)
      .withColumnRenamed("gde_kt", "Wohnkanton")

      // Geburtsjahr: LeiPPos.LPP_LPK_ID → LeiPKopf.LPK_VER_NR → Versicherter.VER_GEB_DAT
      .withColumnRenamed("ver_geb_dat", "Geburtsjahr")

      // Geschlecht: LeiPPos.LPP_LPK_ID → LeiPKopf.LPK_VER_NR → Versicherter.VER_SEX_CD
      .withColumnRenamed("ver_sex_cd", "Geschlecht")

      // Spital Aufenthalt:
      // Exakte Ein/Austritte Spital/Reha o.ä. ECP: LeiPPos.LPP_LPK_ID → LeiPKopf.LPK_RECH_NR → ECKopf.ECK_RECH_NR, ~.ECK_ID → ECKopfXtraCaseDetail.EXC_ECK_ID, ~.EXC_BEG_DAT, ~.EXC_END_DAT (korrekte Summierung)
      // Exakte Ein/Austritte Spital/Reha o.ä. Sumex/Secon: LeiPPos.LPP_LPK_ID → LpkCaseDetail.LCD_BEG_DAT,~.LCD_END_DAT 
      // TODO

      // GTIN: LeiPPos.LPP_LPK_ID → LeiPKopf.LPK_RECH_NR → ECKopf.ECK_RECH_NR, ~.ECK_ID → ECDetail.ECD_ECP_ID→ ECProdukt.ECP_PRODUKT_NR (PharmaCode)
      .join(df_ecKopf, $"LPK_RECH_NR" === $"ECK_RECH_NR", join_type)
      .join(df_ecDetail, $"ECK_ID" === $"ECD_ECK_ID", join_type)
      .join(df_ecProdukt, $"ECD_ECP_ID" === $"ECP_ID", join_type)
      .withColumnRenamed("ecp_produkt_nr", "GTIN")
      

      //Output: AHV-Nummer ,Geburtsjahr, Geschlecht, Wohnkanton, Aufenthalt, GTIN-Code, Anzahl Packungen, Versicherungsmonate, Bruttokosten, Kostenbeteiligung
      .select("AHV_Nr", "Geburtsjahr", "Geschlecht", "Wohnkanton", "GTIN")

df.printSchema()

root
 |-- AHV_Nr: string (nullable = true)
 |-- Geburtsjahr: timestamp (nullable = true)
 |-- Geschlecht: string (nullable = true)
 |-- Wohnkanton: string (nullable = true)
 |-- GTIN: string (nullable = true)



In [6]:
spark.table("default.btl_RA_data").count()

8823944

In [4]:
spark.table("default.btl_RA_data").show()

+-------------+-------------------+----------+----------+-------------+
|       AHV_Nr|        Geburtsjahr|Geschlecht|Wohnkanton|         GTIN|
+-------------+-------------------+----------+----------+-------------+
|7561879110908|1961-12-23 00:00:00|         F|        ZH|   01.01.1100|
|7561879110908|1961-12-23 00:00:00|         F|        ZH|   01.01.1000|
|7561879110908|1961-12-23 00:00:00|         F|        ZH|      1740.00|
|7561879110908|1961-12-23 00:00:00|         F|        ZH|      1372.00|
|7561879110908|1961-12-23 00:00:00|         F|        ZH|7680384230311|
|7561879110908|1961-12-23 00:00:00|         F|        ZH|      39.3005|
|7561879110908|1961-12-23 00:00:00|         F|        ZH|      22.0030|
|7561879110908|1961-12-23 00:00:00|         F|        ZH|      00.0716|
|7561879110908|1961-12-23 00:00:00|         F|        ZH|      00.0610|
|7561879110908|1961-12-23 00:00:00|         F|        ZH|      00.0030|
|7561879110908|1961-12-23 00:00:00|         F|        ZH|      0